In [2]:
import pandas as pd
from sqlalchemy import create_engine, text
import sys
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

# Funções

In [3]:
def connect_to_database(database):
    try:
        connection_string = f'mssql+pyodbc://localhost/{database}?trusted_connection=yes&driver=ODBC+Driver+17+for+SQL+Server'
        engine = create_engine(connection_string)
        return engine 
    except Exception as e:
        print("Erro ao conectar ao banco de dados:", e)
        return None

Função para subir dados para o banco verificam se eles já existem pela chave

In [4]:
def send_to_db(engine, table, pk_column, dataset): 
    if not engine: 
        print('conexão fail')
        sys.exit()

    with engine.connect() as connection:
        for i, row in dataset.iterrows():
            query = text(f"select count(*) from {table} WHERE {pk_column} = :value")
            result = connection.execute(query, parameters=dict(value=row[pk_column]))
            count = result.scalar()

            if count > 0:
                print(f'Registro com {pk_column} = {row[pk_column]} já existe.')
            else: 
                row.to_frame().T.to_sql(table, con=connection, if_exists='append', index=False)
                print(f"Registro com {pk_column} = {row[pk_column]} inserido com sucesso.")
        connection.commit()
        connection.close()

In [5]:
def read_csv(file_path):
    dataset = pd.read_csv(file_path, sep='\t')

    return dataset

# Tabelas do banco

## Tb Account

In [6]:
file_path = r'C:\Users\gabri\OneDrive\Documentos\Projetos\Instagram_data\concorrentes\silver\TbCabecalho.csv'
dataset = read_csv(file_path)

select_columns = [
    'id_account',
    'account_username',
    'account_biography',
    'profile_picture_url',
    'account_name',
    'followers_count', 
    'follows_count', 
    'media_count'
]
dataset= dataset[select_columns]


d_type = {
    'id_account': object
}
dataset = dataset.astype(d_type)
dataset['definicao'] = 'SEARCH'

engine = connect_to_database('DW')
table = 'TbAccount'
pk_column = 'id_account'
send_to_db(engine, table, pk_column, dataset)


Registro com id_account = 17841404384819471 inserido com sucesso.


## Midias

In [7]:
file_path = r'C:\Users\gabri\OneDrive\Documentos\Projetos\Instagram_data\concorrentes\silver\TbMedias.csv'
dataset = pd.read_csv(file_path, sep='\t')

## Aproveitar carregamento para subir FTDMidias
FTbMidias = dataset

select_columns = [
    'id_midia',
    'id_account',
    'media_type',
    'media_url',
    'caption',
    'timestamp',
    'permalink',
    'media_product_type',
    'thumbnail_url'
]
DTbMidias = dataset[select_columns]

# Primeiro, vamos converter as colunas de data para o tipo de dados datetime64
DTbMidias['timestamp'] = pd.to_datetime(DTbMidias['timestamp'])
# # Agora, vamos extrair apenas a data (sem a hora) das colunas de data
# DTbMidias['timestamp'] = DTbMidias['timestamp'].dt.date
type_columns = {
    'id_midia':  'object',
    'id_account': 'object',
}
DTbMidias = DTbMidias.astype(type_columns)

engine = connect_to_database('DW')
table = 'DTbMidias'
pk_column = 'id_midia'
send_to_db(engine, table, pk_column, DTbMidias)

C:\Users\gabri\AppData\Local\Temp\ipykernel_17136\2967277736.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DTbMidias['timestamp'] = pd.to_datetime(DTbMidias['timestamp'])


Registro com id_midia = 17999328089373219 inserido com sucesso.
Registro com id_midia = 18020984945218375 inserido com sucesso.
Registro com id_midia = 18026044757116317 inserido com sucesso.
Registro com id_midia = 17999548157612924 inserido com sucesso.
Registro com id_midia = 18434918029040696 inserido com sucesso.
Registro com id_midia = 18064835116489359 inserido com sucesso.
Registro com id_midia = 18046333051696968 inserido com sucesso.
Registro com id_midia = 17968610261717429 inserido com sucesso.
Registro com id_midia = 18078921748477767 inserido com sucesso.
Registro com id_midia = 18060552868567602 inserido com sucesso.
Registro com id_midia = 18050455996640567 inserido com sucesso.
Registro com id_midia = 17969286797604392 inserido com sucesso.
Registro com id_midia = 17999847821560176 inserido com sucesso.
Registro com id_midia = 17995497869632175 inserido com sucesso.
Registro com id_midia = 17945509649699245 inserido com sucesso.
Registro com id_midia = 1802232332916525

## TbCarrossel

In [8]:
file_path = r'C:\Users\gabri\OneDrive\Documentos\Projetos\Instagram_data\concorrentes\silver\TbCarrossel.csv'
dataset = pd.read_csv(file_path, sep='\t', dtype={'id_midia': str})
rename_columns = {
    'media_type.1': 'type'
}
dataset.rename(columns=rename_columns, inplace=True)
dataset.drop(columns=('Unnamed: 0'),inplace=True)
dataset['id'] = dataset['id'].astype(str) + dataset['id_midia'].astype(str)
dataset.dropna(subset=('id_midia'), inplace=True)

table = 'DTbcarrossel'
pk_column = 'id'
engine = connect_to_database('dw')
send_to_db(engine, table, pk_column, dataset)

Registro com id = 1792781695178566517999328089373219 inserido com sucesso.
Registro com id = 1792781695178566518020984945218375 inserido com sucesso.
Registro com id = 1792781695178566518026044757116317 inserido com sucesso.
Registro com id = 1792781695178566518060552868567602 inserido com sucesso.
Registro com id = 1792781695178566518025158935120064 inserido com sucesso.


C:\Users\gabri\AppData\Local\Temp\ipykernel_17136\128530510.py:15: UserWarning: The provided table name 'DTbcarrossel' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  row.to_frame().T.to_sql(table, con=connection, if_exists='append', index=False)
C:\Users\gabri\AppData\Local\Temp\ipykernel_17136\128530510.py:15: UserWarning: The provided table name 'DTbcarrossel' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  row.to_frame().T.to_sql(table, con=connection, if_exists='append', index=False)
C:\Users\gabri\AppData\Local\Temp\ipykernel_17136\128530510.py:15: UserWarning: The provided table name 'DTbcarrossel' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  row.to_frame().T.to_sql(tab

,id_midia,media_type,type,media_url,id
0,17999328089373219,CAROUSEL_ALBUM,IMAGE,https://scontent.cdninstagram.com/v/t51.29350-...,1792781695178566517999328089373219
1,18020984945218375,CAROUSEL_ALBUM,IMAGE,https://scontent.cdninstagram.com/v/t51.29350-...,1792781695178566518020984945218375
2,18026044757116317,CAROUSEL_ALBUM,IMAGE,https://scontent.cdninstagram.com/v/t51.29350-...,1792781695178566518026044757116317
3,18060552868567602,CAROUSEL_ALBUM,IMAGE,https://scontent.cdninstagram.com/v/t51.29350-...,1792781695178566518060552868567602
4,18025158935120064,CAROUSEL_ALBUM,IMAGE,https://scontent.cdninstagram.com/v/t51.29350-...,1792781695178566518025158935120064


## FTbMidia

In [12]:
FTbMidias.head()

,Unnamed: 0,id_account,username,username.1,id_midia,comments_count,like_count,media_type,media_url,caption,timestamp,permalink,media_product_type,thumbnail_url,extract_date,period,year,day,id_tb_midia
0,0,17841404384819471,gabgalani,raentretenimento,17999328089373219,0,NaN,CAROUSEL_ALBUM,https://scontent.cdninstagram.com/v/t51.29350-...,O fim de semana chegou ao fim mas daqui a pouc...,2024-05-26 22:32:19+00:00,https://www.instagram.com/p/C7ctTWnuea_/,FEED,NaN,2024-05-27,5,2024,27,1799932808937321920240527
1,1,17841404384819471,gabgalani,raentretenimento,18020984945218375,1,NaN,CAROUSEL_ALBUM,https://scontent.cdninstagram.com/v/t51.29350-...,Sextou na Cidade Maravilhosa e tem duas boas f...,2024-05-24 20:49:59+00:00,https://www.instagram.com/p/C7XYADvOV9F/,FEED,NaN,2024-05-27,5,2024,27,1802098494521837520240527
2,2,17841404384819471,gabgalani,raentretenimento,18026044757116317,6,30.0,CAROUSEL_ALBUM,https://scontent.cdninstagram.com/v/t51.29350-...,Fim de semana chegando e é claro que aqui você...,2024-05-23 23:26:21+00:00,https://www.instagram.com/p/C7VFGdZOQa1/,FEED,NaN,2024-05-27,5,2024,27,1802604475711631720240527
3,3,17841404384819471,gabgalani,raentretenimento,17999548157612924,1,NaN,VIDEO,https://scontent.cdninstagram.com/o1/v/t16/f1/...,A melhor festa de pagode de Caraíva desembarca...,2024-05-23 20:59:23+00:00,https://www.instagram.com/reel/C7U0DMEOzbR/,REELS,https://scontent.cdninstagram.com/v/t51.29350-...,2024-05-27,5,2024,27,1799954815761292420240527
4,4,17841404384819471,gabgalani,raentretenimento,18434918029040696,0,NaN,VIDEO,NaN,"O Churrasquinho Menos é Mais, está chegando pa...",2024-05-22 23:25:56+00:00,https://www.instagram.com/reel/C7SgE26ueg3/,REELS,https://scontent.cdninstagram.com/v/t51.29350-...,2024-05-27,5,2024,27,1843491802904069620240527


In [18]:
select_columns = [
    'id_midia',
    'id_account',
    'comments_count',
    'like_count',
    'extract_date',
    'period',
    'year',
    'day'
]

FTbMidias__ = FTbMidias[select_columns]
FTbMidias__['like_count'] = FTbMidias__['like_count'].fillna(0)


FTbMidias__['id'] = FTbMidias__['id_midia'].astype(str) + \
    FTbMidias__['id_account'].astype(str) + FTbMidias__['year'].astype(str) \
          + FTbMidias__['period'].astype(str) + FTbMidias__['day'].astype(str)
FTbMidias__ = FTbMidias__[['id','id_midia', 'id_account', 'comments_count', 'like_count', 'extract_date', 'period', 'year', 'day']]

FTbMidias__['extract_date'] = pd.to_datetime(FTbMidias__['extract_date'])
FTbMidias__['extract_date'] = FTbMidias__['extract_date'].dt.date

type_columns = {
    'id_midia': 'object',
    'id_account': 'object',
    'comments_count': 'int',
    'like_count': 'int',
    'period': 'object',
    'year': 'object',
    'day': 'object'
}

FTbMidias__ = FTbMidias__.astype(type_columns)
FTbMidias__.head()

engine = connect_to_database('DW')
table = 'FTbMidias'
pk_column = 'id'
send_to_db(engine, table, pk_column, FTbMidias__)

C:\Users\gabri\AppData\Local\Temp\ipykernel_17136\2836877084.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  FTbMidias__['like_count'] = FTbMidias__['like_count'].fillna(0)
C:\Users\gabri\AppData\Local\Temp\ipykernel_17136\2836877084.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  FTbMidias__['id'] = FTbMidias__['id_midia'].astype(str) + \


Registro com id = 17999328089373219178414043848194712024527 inserido com sucesso.
Registro com id = 18020984945218375178414043848194712024527 inserido com sucesso.
Registro com id = 18026044757116317178414043848194712024527 inserido com sucesso.
Registro com id = 17999548157612924178414043848194712024527 inserido com sucesso.
Registro com id = 18434918029040696178414043848194712024527 inserido com sucesso.
Registro com id = 18064835116489359178414043848194712024527 inserido com sucesso.
Registro com id = 18046333051696968178414043848194712024527 inserido com sucesso.
Registro com id = 17968610261717429178414043848194712024527 inserido com sucesso.
Registro com id = 18078921748477767178414043848194712024527 inserido com sucesso.
Registro com id = 18060552868567602178414043848194712024527 inserido com sucesso.
Registro com id = 18050455996640567178414043848194712024527 inserido com sucesso.
Registro com id = 17969286797604392178414043848194712024527 inserido com sucesso.
Registro com id 